In [140]:
import pandas as pd
df = pd.read_csv('results_with_crew.csv')

In [ ]:
import http.client
import json
from urllib.parse import quote

def add_new_movie(movie_id, current_df):
    conn = http.client.HTTPSConnection("imdb236.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': "74af810398msh57ffd38221bc0b0p1d3666jsnc105598ca135",
        'x-rapidapi-host': "imdb236.p.rapidapi.com"
    }
    
    conn.request("GET", f"/api/imdb/{quote(movie_id)}", headers=headers)
    res = conn.getresponse()
    data = res.read()
    response_data = json.loads(data.decode("utf-8"))

    if 'error' in response_data:
        print(f"Error from API: {response_data.get('error')}")
        print(f"Could not find movie with ID: {movie_id}. DataFrame not changed.")
        return current_df

    new_df = pd.json_normalize(response_data)
    print("Movie found, adding to DataFrame...")
    
    updated_df = pd.concat([current_df, new_df.rename(columns={'id': 'tconst', 'url': 'IMDbLink'})], ignore_index=True)
    return updated_df


In [ ]:
mov_id = input("Enter your movie ID (e.g., tt0133093): ")
df = add_new_movie(mov_id, df)

searched = df[df['tconst'] == mov_id]
if searched.empty:
    print(f"Movie with ID {mov_id} not found in DataFrame. Adding it now...")
    df = add_new_movie(mov_id, df)
    movie_title_string = searched['primaryTitle'].iloc[0]
else:
    movie_title_string = searched['primaryTitle'].iloc[0]
    print(f"Movie '{movie_title_string}' already exists in the DataFrame.")

Movie found, adding to DataFrame...
Movie 'Spider-Man: No Way Home' already exists in the DataFrame.

DataFrame tail:
          tconst             primaryTitle  startYear    rank  averageRating  numVotes  runtimeMinutes                                          directors                                            writers                        genres  ... countriesOfOrigin                            externalLinks spokenLanguages filmingLocations                                productionCompanies       budget grossWorldwide isAdult metascore                                               cast
4996   tt0122718           Small Soldiers       1998  4997.0            6.3    109720             110                                          Joe Dante  Gavin Scott, Adam Rifkin, Ted Elliott, Terry R...     Action, Adventure, Comedy  ...               NaN                                      NaN             NaN              NaN                                                NaN          NaN         

In [143]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df['genres'] = df['genres'].fillna('')
df['directors'] = df['directors'].fillna('')
df['writers'] = df['writers'].fillna('')

In [144]:
def clean_names(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return [str.lower(i.replace(" ", "")) for i in x.split(',')]
        else:
            return []

In [ ]:
df['directors_clean'] = df['directors'].astype(str).apply(clean_names)
df['writers_clean'] = df['writers'].astype(str).apply(clean_names)
df['genres_clean'] = df['genres'].astype(str).apply(lambda x: [str.lower(i.replace(" ", "")) for i in x.split(',')])

def create_soup(x):
    return ' '.join(x['writers_clean']) + ' ' + ' '.join(x['directors_clean']) + ' ' + ' '.join(x['genres_clean'])

df['features_soup'] = df.apply(create_soup, axis=1)

Feature Soup for first movie:
0    stephenking frankdarabont frankdarabont drama
Name: features_soup, dtype: object


In [146]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['features_soup'])
print("Shape of TF-IDF Matrix:")
print(tfidf_matrix.shape)

Shape of TF-IDF Matrix:
(5001, 8427)


In [147]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("Shape of Cosine Similarity Matrix:")
print(cosine_sim.shape)

Shape of Cosine Similarity Matrix:
(5001, 5001)


In [ ]:
indices = pd.Series(df.index, index=df['primaryTitle']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim, data=df):
    if title not in indices:
        return f"Error: Movie '{title}' not found in the dataset."
    
    idx = indices[title]
    indices_series = indices[title]
    if isinstance(indices_series, pd.Series):
        idx = indices_series.iloc[0]
    else:
        idx = indices_series
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return data['primaryTitle'].iloc[movie_indices]

In [149]:
print(f"Recommendations for '{movie_title_string}':")
print(get_recommendations(movie_title_string))

Recommendations for 'Spider-Man: No Way Home':
1349                      Spider-Man: Far from Home
1345                         Spider-Man: Homecoming
3231    Doctor Strange in the Multiverse of Madness
4611                                          Wolfs
2851                           Ant-Man and the Wasp
1343                                     Spider-Man
4350                                        Cop Car
1073                                 Doctor Strange
2849                 Jumanji: Welcome to the Jungle
3226                         The Amazing Spider-Man
Name: primaryTitle, dtype: object


C:\Users\Savero Madajaya\AppData\Local\Temp\ipykernel_2372\112805545.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = idx[0]
